# 01/20

%matplotlib inline

In [1]:
description = 'mobilenet_from_face_pretrain_sep_feat'

In [2]:

import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from random import shuffle
import math

import pandas as pd

import pickle

from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier,ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing

#from scipy.misc import imread, imresize



In [3]:
import tqdm
import json

In [4]:
train_df = pd.read_csv('../../data/Manually_Annotated_file_lists/training_face_mesh_crop.csv')
train_df.subDirectory_filePath = '../../data/Manually_Annotated_Images_FaceMesh_Cropped/' + train_df.subDirectory_filePath

#train_df_2 = pd.read_csv('../../data/Automatically_annotated_file_list/automatically_annotated_face_mesh_crop.csv')
#train_df_2.subDirectory_filePath = '../../data/Automatically_Annotated_Images_FaceMesh_Cropped/' + train_df_2.subDirectory_filePath
#train_df = train_df.append(train_df_2)
#del train_df_2

train_df = train_df[train_df['have_facemesh']]

In [5]:
from eason_utils import DataFrameBatchIterator
from eason_utils import lprint, now_time_string, log_file_name, change_log_file_name

change_log_file_name(f'''{log_file_name.replace('.log', '')}_{description}.log''')

log_file_name='01_20_22:14:54.log'


In [6]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
config = tf.compat.v1.ConfigProto(gpu_options = tf.compat.v1.GPUOptions(allow_growth = True))
sess = tf.compat.v1.Session(config = config)

In [7]:
mobilenet_pretrained = tf.keras.models.load_model('../models/pretrained_faces/age_gender_tf2_224_deep-03-0.13-0.97.h5')

In [8]:
mobilenet_output = mobilenet_pretrained.get_layer("global_pooling").output

In [9]:
hidden_layers = [256]

In [10]:
valaence_feat = mobilenet_output
for size in hidden_layers:
    valence_feat = tf.keras.layers.Dense(size, activation = 'relu', name = f'feat_valence')(valaence_feat)
outputs_valence = tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'outputs_valence')(valence_feat)

arousal_feat = mobilenet_output
for size in hidden_layers:
    arousal_feat = tf.keras.layers.Dense(size, activation = 'relu', name = f'feat_arousal')(arousal_feat)
outputs_arousal = tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'outputs_arousal')(arousal_feat)

emotion_feat = mobilenet_output
for size in hidden_layers:
    emotion_feat = tf.keras.layers.Dense(size, activation = 'relu', name = f'feat_emotion')(emotion_feat)
emotions_count = len(np.unique(train_df.expression))
outputs_emotion = tf.keras.layers.Dense(emotions_count, activation = 'softmax', name = 'outputs_emotion')(emotion_feat)

In [11]:
model = tf.keras.Model(inputs=mobilenet_pretrained.input,
                       outputs=(outputs_valence, outputs_arousal, outputs_emotion) , name="mobilenet_train")

In [12]:
model.summary()

Model: "mobilenet_train"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 225, 225, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1 (Conv2D)                 (None, 112, 112, 32  864         ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                    

In [13]:
# Instantiate an optimizer.
optimizer = tf.keras.optimizers.Adam()
# Instantiate a loss function.
MSE_loss = tf.keras.losses.MeanSquaredError()
SCC_loss = tf.keras.losses.SparseCategoricalCrossentropy()

In [14]:
batch_size = 32
epochs = 16

logs = []

In [ ]:
from telegram_notifier import send_message
import time
start_time = time.time()


for epoch in range(epochs):
    total_loss = 0
    total_step = 0
    total_emotion_correct = 0
    total_valence_loss = 0
    total_arousal_loss = 0
    total_emotion_loss = 0
    total_process = 0
    for step, row in enumerate(DataFrameBatchIterator(train_df, batch_size=batch_size)):

        imgs = row.subDirectory_filePath.apply(lambda x: cv2.resize(
            cv2.cvtColor(cv2.imread(x), cv2.COLOR_BGR2RGB), (224, 224)))
        img_array = np.array(list(imgs))

        y_valence = np.array(row.valence)
        y_arousal = np.array(row.arousal)
        y_emotion = np.array(row.expression)

        with tf.GradientTape() as tape:
            logits = model(img_array, training=True)

            pred_valence = logits[0]
            pred_arousal = logits[1]
            pred_emotion = logits[2]

            valence_loss = MSE_loss(y_valence, pred_valence)
            arousal_loss = MSE_loss(y_arousal, pred_arousal)
            emotion_loss = SCC_loss(y_emotion, pred_emotion)

            loss = valence_loss + arousal_loss + emotion_loss

            grads = tape.gradient(loss, model.trainable_weights)

            optimizer.apply_gradients(zip(grads, model.trainable_weights))
        total_loss += float(loss)
        total_step += 1
        
        total_process += len(row)

        valence_loss = float(valence_loss)
        arousal_loss = float(arousal_loss)
        emotion_loss = float(emotion_loss)
        
        total_valence_loss += valence_loss
        total_arousal_loss += arousal_loss
        total_emotion_loss += emotion_loss
        
        emotion_correct = int(sum(pred_emotion.numpy().argmax(axis = 1) == y_emotion))
        total_emotion_correct += emotion_correct
        
        log = {
            'emotion_correct': emotion_correct,
            'valence_loss': valence_loss,
            'arousal_loss': arousal_loss,
            'emotion_loss': emotion_loss,
            'time_take': time.time() - start_time
        }
        log = json.dumps(log)
        lprint(log)
    save_model_path = f"models/{now_time_string}_{description}_epoch{epoch}_batch_{batch_size}"
    model.save(save_model_path)
    lprint(f"Save {save_model_path}")
    send_message(f"Save {save_model_path}\nepoch {epoch} is finish")
    log = {
        'model_path': save_model_path,
        'total_emotion_correct': total_emotion_correct,
        'total_loss': total_loss,
        'total_valence_loss': total_valence_loss,
        'total_arousal_loss': total_arousal_loss,
        'total_emotion_loss': total_emotion_loss,
        'total_process': total_process,
        'time_take': time.time() - start_time
    }
    log = json.dumps(log)
    lprint(log)

{"emotion_correct": 2, "valence_loss": 2.756927490234375, "arousal_loss": 2.5483057498931885, "emotion_loss": 2.3268074989318848, "time_take": 3.7601218223571777}
{"emotion_correct": 6, "valence_loss": 1.0605723857879639, "arousal_loss": 0.910541296005249, "emotion_loss": 2.2088069915771484, "time_take": 4.083930730819702}
{"emotion_correct": 11, "valence_loss": 1.5213910341262817, "arousal_loss": 1.3364959955215454, "emotion_loss": 1.9057841300964355, "time_take": 4.290806293487549}
{"emotion_correct": 13, "valence_loss": 1.1198079586029053, "arousal_loss": 0.955219030380249, "emotion_loss": 1.7774360179901123, "time_take": 4.601081848144531}
{"emotion_correct": 16, "valence_loss": 0.8867492079734802, "arousal_loss": 0.6715497374534607, "emotion_loss": 1.52064847946167, "time_take": 4.919289588928223}
{"emotion_correct": 17, "valence_loss": 0.8881931304931641, "arousal_loss": 0.7615321278572083, "emotion_loss": 1.6821653842926025, "time_take": 5.282388687133789}
{"emotion_correct": 14

{"emotion_correct": 15, "valence_loss": 1.1764615774154663, "arousal_loss": 1.1205761432647705, "emotion_loss": 1.5504088401794434, "time_take": 18.099183320999146}
{"emotion_correct": 17, "valence_loss": 0.8973537683486938, "arousal_loss": 0.7201874852180481, "emotion_loss": 1.3589215278625488, "time_take": 18.41706418991089}
{"emotion_correct": 13, "valence_loss": 1.0065486431121826, "arousal_loss": 0.8076820373535156, "emotion_loss": 1.6351312398910522, "time_take": 18.6439425945282}
{"emotion_correct": 17, "valence_loss": 1.0528804063796997, "arousal_loss": 0.9171227812767029, "emotion_loss": 1.2168645858764648, "time_take": 18.918928861618042}
{"emotion_correct": 12, "valence_loss": 0.8614283204078674, "arousal_loss": 0.7409688234329224, "emotion_loss": 1.7003986835479736, "time_take": 19.237633228302002}
{"emotion_correct": 18, "valence_loss": 1.300057053565979, "arousal_loss": 1.0828254222869873, "emotion_loss": 1.4850261211395264, "time_take": 19.518555402755737}
{"emotion_corr

{"emotion_correct": 20, "valence_loss": 0.5947421789169312, "arousal_loss": 0.4883272647857666, "emotion_loss": 1.3004716634750366, "time_take": 31.289317846298218}
{"emotion_correct": 17, "valence_loss": 1.1196165084838867, "arousal_loss": 0.9686561226844788, "emotion_loss": 1.488154649734497, "time_take": 31.53326153755188}
{"emotion_correct": 14, "valence_loss": 1.186267614364624, "arousal_loss": 1.0787780284881592, "emotion_loss": 1.5496225357055664, "time_take": 31.80396342277527}
{"emotion_correct": 23, "valence_loss": 1.1028962135314941, "arousal_loss": 0.9521522521972656, "emotion_loss": 1.1434223651885986, "time_take": 32.02362012863159}
{"emotion_correct": 13, "valence_loss": 0.8519669771194458, "arousal_loss": 0.7446374893188477, "emotion_loss": 1.6431982517242432, "time_take": 32.384533405303955}
{"emotion_correct": 14, "valence_loss": 1.010996699333191, "arousal_loss": 0.9469037652015686, "emotion_loss": 1.718285083770752, "time_take": 32.646446228027344}
{"emotion_correct

{"emotion_correct": 19, "valence_loss": 1.0708295106887817, "arousal_loss": 0.9807083010673523, "emotion_loss": 1.6538755893707275, "time_take": 45.72609996795654}
{"emotion_correct": 20, "valence_loss": 1.3494195938110352, "arousal_loss": 1.2201205492019653, "emotion_loss": 1.232127070426941, "time_take": 46.01568841934204}
{"emotion_correct": 22, "valence_loss": 0.7703313827514648, "arousal_loss": 0.5751356482505798, "emotion_loss": 1.086447834968567, "time_take": 46.39025616645813}
{"emotion_correct": 19, "valence_loss": 1.2944364547729492, "arousal_loss": 1.0416874885559082, "emotion_loss": 1.0563983917236328, "time_take": 46.699623107910156}
{"emotion_correct": 20, "valence_loss": 0.9688553810119629, "arousal_loss": 0.8234134912490845, "emotion_loss": 1.2940974235534668, "time_take": 46.95645022392273}
{"emotion_correct": 20, "valence_loss": 0.8472476005554199, "arousal_loss": 0.6366068124771118, "emotion_loss": 1.2390143871307373, "time_take": 47.227752685546875}
{"emotion_correc

{"emotion_correct": 19, "valence_loss": 1.0173518657684326, "arousal_loss": 0.823983907699585, "emotion_loss": 1.3129957914352417, "time_take": 61.149840354919434}
